In [2]:
import pandas as pd
import pulp

In [5]:
stock_df = pd.read_csv('stocks.csv')
stock_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [6]:
require_df = pd.read_csv('requires.csv')
require_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [7]:
gain_df = pd.read_csv('gains.csv')
gain_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [8]:
P = gain_df['p'].tolist()
print(P)

['p1', 'p2', 'p3', 'p4']


In [10]:
M = stock_df['m'].tolist()
print(M)

['m1', 'm2', 'm3']


In [11]:
stock = {row.m:row.stock for row in stock_df.itertuples()}
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [12]:
stock = dict(zip(stock_df['m'], stock_df['stock']))
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [13]:
stock = dict((row.m, row.stock) for row in stock_df.itertuples())
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [14]:
stock = stock_df.set_index('m').to_dict()['stock']
print(stock)

{'m1': 35, 'm2': 22, 'm3': 27}


In [15]:
require = {(row.p, row.m):row.require for row in require_df.itertuples()}
print(require)

{('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}


In [16]:
gain = {row.p:row.gain for row in gain_df.itertuples()}
print(gain)

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}


In [17]:
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

In [29]:
x = pulp.LpVariable.dicts('x', P, cat='Continuous')

In [19]:
x = {}
for p in P:
  x[p] = pulp.LpVariable('x={}'. format(p), cat='Continuous')

In [20]:
x = {}
for p in P:
  x[p] = pulp.LpVariable(f'x_{p}', cat='Continuous')

In [21]:
x = {p:pulp.LpVariable(f'x_{p}', cat='Continuous') for p in P}

In [22]:
for p in P:
  problem += x[p] >= 0

In [23]:
for m in M:
  problem += pulp.lpSum([require[p,m]*x[p] for p in P]) <= stock[m]

In [25]:
problem += pulp.lpSum([gain[p]*x[p] for p in P])

In [26]:
status = problem.solve()
print('Status:', pulp.LpStatus[status])

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --cpxlp /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/421eedf9ed5a4d01a05efca1a26e4267-pulp.lp
 -o /var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/421eedf9ed5a4d01a05efca1a26e4267-pulp.sol
Reading problem data from '/var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/421eedf9ed5a4d01a05efca1a26e4267-pulp.lp'...
7 rows, 4 columns, 13 non-zeros
17 lines were read
GLPK Simplex Optimizer, v4.65
7 rows, 4 columns, 13 non-zeros
Preprocessing...
3 rows, 4 columns, 9 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.000e+00  ratio =  3.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (4)
*     4: obj =   8.042857143e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40424 bytes)
Writing basic solution to '/var/folders/f1/dg38sdvs7tsgxf3hbspgk1vc0000gn/T/421

In [27]:
for p in P:
  print(p, x[p].value())

print('obj=', problem.objective.value())

p1 12.1429
p2 3.57143
p3 7.42857
p4 0.0
obj= 80.42869999999999
